In [ ]:
import pandas as pd
import numpy as np

In [ ]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/data.csv', names=r_cols, encoding='latin-1')

FileNotFoundError: ignored

In [ ]:
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1) # timestamp열 제거(axis=1로 해서 열제거)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

In [ ]:
# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy() # x는 ratings데이터를 복사한것(주소말고 데이터?를)
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y) # stratify는 user_id를 기준해서 데이터를 나눠주겠다. 위에서 y에 user_id를 할당했으니까

In [ ]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [ ]:
# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 리스트컴프리헨션인듯?
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [ ]:
# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix

In [ ]:
# Full matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id')

#### 성별 추천 코드를 수정해서 사용자의 직업('occupation')에 따라 집단을 나누어서 예측값을 구하는 함수를 만들고 이의 정확도를 계산하는 코드를 작성하세요

In [ ]:
# occupation별 평점평균 계산
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

In [ ]:
###### Gender기준 추천 ######
# gender별 평균을 예측치로 돌려주는 함수 
def cf_occupation(user_id, movie_id):
    if movie_id in rating_matrix:
        occu = users.loc[user_id]['occupation']
        
        if occu in o_mean[movie_id]:
            occu_rating = o_mean[movie_id][occu]
        else:
            occu_rating = 3.0
    else:
        occu_rating = 3.0
    return occu_rating

score(cf_occupation)

#### 사용자의 성별과 직업('occupation')을 동시에 고려한 집단을 나누어서 예측값을 구하는 함수를 만드리고 이의 정확도를 계산하는 코드를 작성하세요

In [ ]:
# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [ ]:
# occupation별 평점평균 계산
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

In [ ]:
###### Gender기준 추천 ######
# gender별 평균을 예측치로 돌려주는 함수 
def cf_occupation_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        occu = users.loc[user_id]['occupation']
        if occu in o_mean[movie_id]:
            occu_rating = o_mean[movie_id][occu]
        else:
            occu_rating = 3.0
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        occu_rating = 3.0
        gender_rating = 3.0
    occu_sex_rating = (occu_rating+gender_rating)/2
    return occu_sex_rating

score(cf_occupation_gender)